### 3 Assignment
### 다음 미완성 코드를 활용해 좀 더 긴 문장을 학습해보자
* Sample sentences
  + "if you want to build a ship, don't drum up people together to "
  + "collect wood and don't assign them tasks and work, but rather "
  + "teach them to long for the endless immensity of the sea."
* Training data sentence
  + Shape: (N, S, E)
* Hidden
  + Shape: (N, S, E *2)
* Output
  + Shape: (N, S, E)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


# for reproducibility
torch.manual_seed(100)

# Dictionary
sample_sentence_1 = "if you want to build a ship, don't drum up people together to "
sample_sentence_2 = "collect wood and don't assign them tasks and work, but rather "
sample_sentence_3 = "teach them to long for the endless immensity of the sea."
sample_sentence = sample_sentence_1 + sample_sentence_2 + sample_sentence_3
sample_sentences = [sample_sentence_1, sample_sentence_2, sample_sentence_3]
char_set = list(set(sample_sentence))
dic = {c: i for i, c in enumerate(char_set)}

# Parameters
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size * 2
output_size = dic_size
unit_sequence_length = 20

# Dataset setting
input_batch = []
target_batch = []

for sentence in sample_sentences:
  for i in range(len(sentence)):
    if i+unit_sequence_length+1 > len(sentence):
      break
      # x_data = [dic[c] for c in sentence[i:-1]]
      # y_data = [dic[c] for c in sentence[i+1:]]

    else:
      x_data = [dic[c] for c in sentence[i:i+unit_sequence_length]]
      y_data = [dic[c] for c in sentence[i+1:i+unit_sequence_length+1]]

    x_one_hot = [np.eye(dic_size)[x] for x in x_data]
    input_batch.append(x_one_hot)
    target_batch.append(y_data)

# To torch tensors
X = torch.FloatTensor(np.array(input_batch))
Y = torch.LongTensor(np.array(target_batch))
print(X.shape)
print(Y.shape)


torch.Size([120, 20, 25])
torch.Size([120, 20])


In [ ]:
""" 
  실습 때 사용한 모델을 기초로 해서, 
  RNN쪽 부분을 RNN, LSTM, GRU 세 가지로 바꾸어가며 테스트해 결과 accuracy를 비교해본다
"""

# Model RNN
class Custom_RNN(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Custom_RNN, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers)
    self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x


In [ ]:
# Model LSTM
class Custom_LSTM(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Custom_LSTM, self).__init__()
    self.lstm = torch.nn.LSTM(input_dim, hidden_dim, num_layers=layers)
    self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.lstm(x)
    x = self.fc(x)
    return x

In [ ]:
# Model GRU
class Custom_GRU(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Custom_GRU, self).__init__()
    self.gru = torch.nn.GRU(input_dim, hidden_dim, num_layers=layers)
    self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.gru(x)
    x = self.fc(x)
    return x

In [ ]:
learning_rate = 0.05
training_epochs = 100
model1 = Custom_RNN(input_size, hidden_size, output_size, 2)
model2 = Custom_LSTM(input_size, hidden_size, output_size, 2)
model3 = Custom_GRU(input_size, hidden_size, output_size, 2)

# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss()    # Softmax
optimizer1 = optim.Adam(model1.parameters(), lr=learning_rate)
optimizer2 = optim.Adam(model2.parameters(), lr=learning_rate)
optimizer3 = optim.Adam(model3.parameters(), lr=learning_rate)

print('*training RNN')
# train RNN
for epoch in range(training_epochs):
  optimizer1.zero_grad()
  outputs1 = model1(X)
  loss = criterion(outputs1.reshape(-1, dic_size), Y.reshape(-1))
  loss.backward()
  optimizer1.step()
  if epoch % 10 == 9:
    print('epoch: ',epoch, 'loss: ', loss.item())

print('*training LSTM')
# train LSTM
for epoch in range(training_epochs):
  optimizer2.zero_grad()
  outputs2 = model2(X)
  loss = criterion(outputs2.reshape(-1, dic_size), Y.reshape(-1))
  loss.backward()
  optimizer2.step()
  if epoch % 10 == 9:
    print('epoch: ',epoch, 'loss: ', loss.item())

print('*training GRU')
# train GRU
for epoch in range(training_epochs):
  optimizer3.zero_grad()
  outputs3 = model3(X)
  loss = criterion(outputs3.reshape(-1, dic_size), Y.reshape(-1))
  loss.backward()
  optimizer3.step()
  if epoch % 10 == 9:
    print('epoch: ',epoch, 'loss: ', loss.item())


# result RNN
correct_prediction = torch.argmax(outputs1, 2) == Y
acc = correct_prediction.float().mean()
print('RNN Accuracy:', acc.item())

# result LSTM
correct_prediction = torch.argmax(outputs2, 2) == Y
acc = correct_prediction.float().mean()
print('LSTM Accuracy:', acc.item())

# result GRU
correct_prediction = torch.argmax(outputs3, 2) == Y
acc = correct_prediction.float().mean()
print('GRU Accuracy:', acc.item())

*training RNN
epoch:  9 loss:  2.907910108566284
epoch:  19 loss:  2.6353232860565186
epoch:  29 loss:  2.494781255722046
epoch:  39 loss:  2.3309452533721924
epoch:  49 loss:  2.3871140480041504
epoch:  59 loss:  2.9361302852630615
epoch:  69 loss:  2.9850499629974365
epoch:  79 loss:  2.656057834625244
epoch:  89 loss:  2.649611711502075
epoch:  99 loss:  2.636887311935425
*training LSTM
epoch:  9 loss:  2.7562332153320312
epoch:  19 loss:  1.8871384859085083
epoch:  29 loss:  0.8456621170043945
epoch:  39 loss:  0.27720290422439575
epoch:  49 loss:  0.11401919275522232
epoch:  59 loss:  0.04747263342142105
epoch:  69 loss:  0.02587020955979824
epoch:  79 loss:  0.018534649163484573
epoch:  89 loss:  0.015526099130511284
epoch:  99 loss:  0.014072778634727001
*training GRU
epoch:  9 loss:  2.3059887886047363
epoch:  19 loss:  1.0155415534973145
epoch:  29 loss:  0.25913941860198975
epoch:  39 loss:  0.08544573187828064
epoch:  49 loss:  0.03231180086731911
epoch:  59 loss:  0.0186889